In [1]:
import torch
import pandas as pd
import numpy as np
from math import log, exp
import math

In [2]:
dataRaw = pd.read_csv('Admission_Predict.csv')
dataRaw.head()
# del dataRaw['Serial No.'] # Serves as a unique identifier, not needed for modeling

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
len(dataRaw)

400

In [309]:
dataRaw.iloc[:,1].min()

290

In [310]:
dataRaw.iloc[:,1].max()

340

In [301]:
len(dataRaw.iloc[0, :])

8

In [9]:
dataRaw.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [10]:
msk =  np.random.rand(len(dataRaw)) < 0.8

In [416]:
msk

array([False, False,  True,  True,  True, False,  True, False,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [11]:
trainData = dataRaw[msk]
testData = dataRaw[~msk]

In [13]:
def normalize(x, m, s):
    return (x-m)/s

In [17]:
msk =  np.random.rand(len(dataRaw)) < 0.8

trainData = dataRaw[msk]
validData = dataRaw[~msk]

In [18]:
x_train = torch.tensor(trainData.iloc[:,:-1].values, dtype=torch.float)
y_train = torch.tensor(trainData.iloc[:,-1:].values, dtype=torch.float)

x_valid = torch.tensor(validData.iloc[:,:-1].values, dtype=torch.float)
y_valid = torch.tensor(validData.iloc[:,-1:].values, dtype=torch.float)

In [348]:
x_train = (x_train-x_train.min())/(x_train.max()-x_train.min())
x_valid = (x_valid-x_valid.min())/(x_valid.max()-x_train.min())

In [349]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1863), tensor(0.3225))

In [350]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [351]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(2.0058e-07), tensor(1.))

In [53]:
class LogisticRegression:
    def __call__(self, y, yhat):
        self.y = y
        self.yHat = yhat
        tmp = (-1)*y*log(max(yhat, 0.00000000000000000001)) - (1-y)*log(max(1-yhat,0.00000000000000000001))
        return tmp
    
    def backward(self, grad):
        '''
        [(-y)/(yHat) - (1-y)/(1-yHat)]
        '''
        left = ((-1)*(self.y))/(self.yHat)
        right = (1-self.y)/(1-self.yHat)
        
        grad *= (left-right)
        
        return grad
        
        

In [466]:
class Sigmoid:
    def __call__(self, x):
        self.x = x
#         print("type of input is: {}".format(type(x)))
#         print("dim of input is: {}".format(x.dim()))
#         print("shape of input is: {}".format(x.shape))
#         print("input is: {}".format(x))
        return 1/(1+torch.exp((-1)*x)) 
    
    def backward(self, grad):
        '''
        (e^-x)/((1+e^-x)^2)
        '''
        top = torch.exp((-1)*self.x)
        bottom = (1+torch.exp((-1)*self.x))**2
        
        grad *= top/bottom
        
        return grad
    
    def refresh(self):
        self.x = 0
        
        
    def update(self):
        pass

In [504]:
class Linear:
    def __init__(self, m, n, alpha):
        self.m = m
        self.n = n
#         self.w = torch.randn(m,n)*math.sqrt(2/m)
        self.w = torch.randn(m,n)
#         self.b = torch.zeros(n)
        self.b = torch.randn(n)
        
        self.lr = alpha
        self.count = 0
        self.nW = 1
        self.nB = 1
        
    def __call__(self, x):
        self.inp = x
        self.out = x@self.w + self.b 
        return x@self.w + self.b
    
    def refresh(self):
        self.w = torch.randn(self.m,self.n)*math.sqrt(2/self.m)
        self.b = torch.zeros(self.n)
        
        self.count = 0
        self.nW = 1
        self.nB = 1
    
    def backward(self, grad):
        self.count += 1
        self.nW += (self.inp.unsqueeze(-1) * grad.unsqueeze(0)).sum(0)
        self.nB += grad.sum(0)

        if grad.dim() == 0:
            grad = grad.unsqueeze(-1)@self.w.t()
            
        elif grad.dim() != 0:
            grad = grad@self.w.t()
            
        return grad
    
    def update(self):
        self.nW /= self.count
        self.nB /= self.count
        
        self.w = self.w.add_( (-1)*(self.lr)*(self.nW) )
        self.b = self.b.add_( (-1)*(self.lr)*(self.nB) )
        
        self.count = 0
        self.nW = 1
        self.nB = 1
        

In [440]:
class Model:
    def __init__(self, layers):
        self.layers = layers
        self.activation = Sigmoid()
        self.loss = LogisticRegression()
        
    def forwards(self, x):
        
        for layer in self.layers:
            x = layer(x) 
        
        yHat = self.activation(x)
        
        self.yHat = yHat
        
        return yHat
    
    def getLoss(self, y):
        return self.loss(y, self.yHat)
    
    def backward(self):
        self.grad = 1
        
        expandedLayers = [self.loss, self.activation]
        
        for layer in expandedLayers:
            self.grad = layer.backward(self.grad)
            
        # Iterate through expanded layers
        # Then iterate through the reverse of linear layers
        for layer in reversed(self.layers):
            self.grad = layer.backward(self.grad)
            
        return self.grad
    
    def update(self):
        for layer in self.layers:
            layer.update()

In [428]:
class Learner:
    def __init__(self, model, x_train, y_train, x_test, y_test, epochs=5):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.epochs = epochs
        
    def train(self):
        loss = 0
        for epoch in range(self.epochs):
            
            for i in range(len(self.x_train)):
#                 loss += self.model.forwards(self.x_train[i], self.y_train[i])

                self.model.forwards(self.x_train[i])
                
                loss += self.model.getLoss(self.y_train[i])
                
                self.model.backward()
                
            print("[loss for epoch {}] {}".format(epoch+1, loss/len(self.x_train)))
            
            loss = 0
            self.model.update()
            
    def train_oneCycle(self):
        loss = 0
        for i in range(len(self.x_train)):
#             loss += self.model.forwards(self.x_train[i], self.y_train[i])
            
            self.model.forwards(self.x_train[i])

            loss += self.model.getLoss(self.y_train[i])
            
            self.model.backward()

        print("[loss for train_oneCycle] {}".format(loss/len(self.x_train)))

        loss = 0
        self.model.update()
        
    def train_xCycles(self, x):
        for j in range(x):
            loss = 0
            for i in range(len(self.x_train)):
#                 loss += self.model.forwards(self.x_train[i], self.y_train[i])
                
                self.model.forwards(self.x_train[i])

                loss += self.model.getLoss(self.y_train[i])
                
                
                self.model.backward()

            print("[loss for {}] {}".format(j, loss/len(self.x_train)))

            loss = 0
            self.model.update()
            
    def validate(self):
        loss = 0
        for i in range(len(self.x_test)):
#             loss += self.model.forwards(self.x_test[i], self.y_test[i])
            
            self.model.forwards(self.x_test[i])

            loss += self.model.getLoss(self.y_test[i])
            
        print("[test loss] {}".format((loss/len(self.x_test))))
        
    def validateRMSE(self):
        '''
        (yHat-y)^2
        ave
        sqrt
        '''
        
        ans = 0
        residuals = 0
        for i in range(len(self.x_test)):
#             loss += self.model.forwards(self.x_test[i], self.y_test[i])
            yHat = self.model.forwards(self.x_test[i])
    
            residuals += yHat-self.y_test[i]
        
        residuals *= residuals
        residuals /= len(self.x_test)
        return math.sqrt(residuals)
    
        
        
    def trainAndValidate(self):
        self.train()
        self.validate()
        
    def refresh(self):
        for l in self.model.layers:
            l.refresh()

In [501]:
alpha = 0.01
# model = Model([Linear(7, 25, alpha), Sigmoid(), Linear(25, 1, alpha)])
model = Model([Linear(7, 25, alpha), Linear(25, 1, alpha)])

In [502]:
learn = Learner(model, x_train, y_train, x_valid, y_valid, epochs=5)

In [503]:
def refreshAndTrainOnce():
    learn.refresh()
    learn.train_oneCycle()

In [498]:
refreshAndTrainOnce()

[loss for train_oneCycle] tensor([0.9742])


In [491]:
learn.train_oneCycle()

[loss for train_oneCycle] tensor([0.7892])


In [407]:
learn.train_xCycles(4)

[loss for 0] tensor([0.6009])
[loss for 1] tensor([0.6080])
[loss for 2] tensor([0.6288])
[loss for 3] tensor([0.6658])


In [124]:
learn.train()

[loss for epoch 1] tensor([0.5971])
[loss for epoch 2] tensor([0.5972])
[loss for epoch 3] tensor([0.5973])
[loss for epoch 4] tensor([0.5976])
[loss for epoch 5] tensor([0.5980])


In [492]:
learn.validate()

[test loss] tensor([0.7799])


In [493]:
learn.validateRMSE()

1.8506973756437328

In [ ]:
'''
RMSE:
sqrt(MSE)
'''